In [ ]:
#justclickrun #easy-peasy

from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
import os
%gui tk
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tifffile import imread
from tkinter import filedialog
from csbdeep.utils import Path, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE
from IPython.display import clear_output
from tqdm import tqdm

root = tk.Tk()
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

chan = int(input("Number of channels: "))
directory = filedialog.askdirectory(title="Choose big folder directory")

models = list()
for i in range(chan):
    i = i + 1
    modeldir = filedialog.askdirectory(title="Choose model " + str(i), initialdir=os.path.dirname("models/"))
    models.append(CARE(config=None, name=str.split(modeldir, "/")[-1], basedir=modeldir + "/../"))

for root, dirs, files in os.walk(directory):
    for i in dirs:
        if i.endswith("tiff"): # Processed folder from step 1
            filecounter = 0
            for x in os.scandir(directory + "/" + i):
                filecounter += 1
                
            for file in tqdm(os.scandir(directory + "/" + i), total=filecounter, unit="files"):
                if file.name.endswith(".tif"):
                    current = imread(file.path)
                    size = current.shape
                    result = np.zeros(size)
                    print(file.path)
                    for c in range(chan):
                        print('Channel ' + str(c+1))
                        result[:,c,:,:] = models[c].predict(current[:,c,:,:], 'ZYX', n_tiles=(1,1,1))
                    Path(directory + "/" + i[:-4] + 'restored').mkdir(exist_ok=True)
                    save_tiff_imagej_compatible(directory + "/" + i[:-4] + 'restored/%s' % file.name, result, 'ZCYX')
                    clear_output(wait=True)
print("Done!")